In [1]:
# accounts
import json
import solana
import time
import sys
import glob
from solana.rpc.api import Client
from solana.blockhash import Blockhash
from solana.publickey import PublicKey
from solana.system_program import CreateAccountWithSeedParams, create_account_with_seed, CreateAccountParams, create_account
from solana.transaction import Transaction, AccountMeta, TransactionInstruction
from solana.system_program import SYS_PROGRAM_ID
from spl.token.client import Token
from solana import keypair
from solana.publickey import PublicKey
from solana.sysvar import SYSVAR_RENT_PUBKEY
from struct import *
import struct

In [2]:
TOKEN_PROGRAM_ID: PublicKey = PublicKey("TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA")

# first_key_in_key_folder = glob.glob("../keys/*")[0]
# payer_loaded_account = solana.keypair.Keypair(json.load(open(first_key_in_key_folder))[:32])
# payer_public_key = payer_loaded_account.public_key
payer_loaded_account = solana.keypair.Keypair.generate()
payer_public_key = payer_loaded_account.public_key



http_client = Client("https://api.devnet.solana.com")




# todo: change program_id to system program, change data size =0, change lampord for both alice and bob
alice_keypair = solana.keypair.Keypair.generate()
alice_pubkey = alice_keypair.public_key



bob_keypair = solana.keypair.Keypair.generate()
bob_pubkey = bob_keypair.public_key


print(f"Payer Account: {payer_public_key}")
print(f"alice Account: {alice_pubkey}")
print(f"bob Account: {bob_pubkey}")

Payer Account: GqQtnnYHYxaPibo2VmqwckwHy6E43NYQ6Gw8Qb9GRJqM
alice Account: B7s8HFofhSWzZrWAFdd59Ww2fynU95jBEUFoADwKGhMX
bob Account: HSMCvhQAsbi6T7dYffsZZ6Gfp5NAQ8PzGbhTVix3QYSu


In [3]:
import time
http_client.request_airdrop(payer_loaded_account.public_key, 1000000000)
time.sleep(15)
http_client.request_airdrop(alice_pubkey, 1000000000)
time.sleep(15)
http_client.request_airdrop(bob_pubkey, 1000000000)
print(http_client.get_balance(alice_pubkey))
# print(http_client.get_balance(bob_pubkey))
# print(http_client.get_balance(payer_loaded_account.public_key))
# print(http_client.get_minimum_balance_for_rent_exemption(7)["result"])

{'jsonrpc': '2.0', 'result': {'context': {'slot': 87131706}, 'value': 1000000000}, 'id': 4}


In [4]:
X_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
Y_mint_account_address = Token.create_mint(conn= http_client, payer = payer_loaded_account, mint_authority = payer_loaded_account.public_key, decimals = 0, program_id = TOKEN_PROGRAM_ID, )
alice_X_token_account = X_mint_account_address.create_associated_token_account(alice_pubkey)
alice_Y_token_account = Y_mint_account_address.create_associated_token_account(alice_pubkey)
bob_X_token_account = X_mint_account_address.create_associated_token_account(bob_pubkey)
bob_Y_token_account = Y_mint_account_address.create_associated_token_account(bob_pubkey)

In [21]:
X_mint_account_address.mint_to(alice_X_token_account,payer_loaded_account,1000)
Y_mint_account_address.mint_to(bob_Y_token_account,payer_loaded_account,100)

{'jsonrpc': '2.0',
 'result': '4DKrhjtakus8X53VDYLqtFWA27uKHoJLBTiTLadQqRi6tYf7gKwjUvMR2pSeofrzazXzmG3pvodf52J7mVBMXP4S',
 'id': 64}

In [22]:
# escrow meta-data account
deployed_program_key = glob.glob("../target/deploy/*.json")[0]
deployed_program_key_account = solana.keypair.Keypair(json.load(open(deployed_program_key))[:32])
program_id = deployed_program_key_account.public_key

In [63]:
# todo, create seed more specific
password = 25
x_seeds = [
    b"vault_x",
    password.to_bytes(8,byteorder='little'),
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]

y_seeds = [
    b"vault_y",
    password.to_bytes(8,byteorder='little'),
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]

escrow_seeds = [
    b"escrow",
    password.to_bytes(8,byteorder='little'),
    bytes(alice_pubkey),
    bytes(bob_pubkey),
    bytes(X_mint_account_address.pubkey),
    bytes(Y_mint_account_address.pubkey),
]


vaultx, xseed = PublicKey.find_program_address(seeds=x_seeds,program_id=program_id)
vaulty, yseed = PublicKey.find_program_address(seeds=y_seeds,program_id=program_id)
escrow_address, escrow_seed = PublicKey.find_program_address(seeds=escrow_seeds,program_id=program_id)

In [ ]:
alice1 = [10, 140, 229, 196, 196, 173, 137, 112, 235, 104, 209, 194, 248, 49, 198, 12, 133, 186, 173, 133, 113, 136, 155, 72, 174, 44, 101, 204, 105, 117, 221, 119]
len(alice1)

In [ ]:
bytes(alice_pubkey)

In [ ]:
(229).to_bytes(1,byteorder='little')

In [ ]:
escrow_address

In [64]:
print(vaultx)
print(vaulty)
print(escrow_address)
print(program_id)

8kJmM91VvRxmsBJogHQfwbX177mWNUDP6p3PYTvhEWLP
3rpMYTD7RhrwSsPTc7qU55GJeyH4yNe2G5Fg8hRMxkFw
BAdFTNUyjP16bcxS4FqixqQ8udjj5wKqFEHE5TE1v5cZ
6rtZCTuF3hwKZhm6R6HJQGQ4nYq6ihYsCKXd4rUBw9KL


In [ ]:
password = (16).to_bytes(8,byteorder='little')
data1 = pack('<BQQ', 0,13,15)  + password
data2 = pack('<BQQQ', 0,13,15,16)
print(data1)
print(data2)

In [ ]:
pass1 = [16, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
pack('<BQQ', 0,13,15)

In [ ]:
bytes(13)

In [52]:
alice_Y_token_account

HPt4ebfceymeUZ8HD2XUYSMzpcqqwmVRDKZWfxo4Y26w

In [ ]:
(14).to_bytes(1,byteorder='little')

In [ ]:
password

In [65]:
# initialize transaction
x_val = 13
y_val = 15
data = pack('<BQQQ', 0,x_val,y_val,password)
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=X_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=Y_mint_account_address.pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=payer_public_key, is_signer=True, is_writable=False),
          AccountMeta(pubkey=alice_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=bob_pubkey, is_signer=False, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYSVAR_RENT_PUBKEY, is_signer=False, is_writable=False),
          AccountMeta(pubkey=SYS_PROGRAM_ID, is_signer=False, is_writable=False),
#           AccountMeta(pubkey=program_id, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=data, #struct.pack("<LHB",1,1,1) # b58encode(bytes([9] * 7))#b'0001000100010001000100010001',
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[payer_loaded_account])#, *[payer_loaded_account])

In [60]:
escrow_address

GLtDmR429wbumeYUWxYkifFS9gTohfDKY9pmaVJg3Jzh

In [66]:
time.sleep(15)
a = http_client.get_account_info(escrow_address)['result']['value']['data'][0]
import base64
print(len(base64.b64decode(a)))
print(unpack('<QQ', base64.b64decode(a)[:16]))
print(unpack('<Q??', base64.b64decode(a)[208:218]))
print(base64.b64decode(a)[16:16+32])
bytes(alice_pubkey)

# Q 8 bytes
# a 400 bytes


# xval: u64,   8 byte
# yval: u64,   8 byte
# a_pub_key: Pubkey, 32 byte
# b_pub_key: Pubkey, 32 byte
# mint_x_pub_key: Pubkey,  32 byte
# mint_y_pub_key: Pubkey,  32 byte
# vault_x_pub_key: Pubkey, 32 byte
# vault_y_pub_key: Pubkey, 32 byte
# init_deposit_status: u64, 8 byte
# is_a_withdrawed: u8,    1byte
# is_b_withdrawed: u8,    1byte

218
(13, 15)
(1, False, False)
b'\x96W\x07\xfeo\x9b\xda>R}\x8b:v\xa1\xbeT\x03\\B7\xee?]\xa6\xd5\x15\xddF\x7fs\xcc\xfe'


b'\x96W\x07\xfeo\x9b\xda>R}\x8b:v\xa1\xbeT\x03\\B7\xee?]\xa6\xd5\x15\xddF\x7fs\xcc\xfe'

In [ ]:
base64.b64decode(a)[:227]

In [67]:
# deposite x
from struct import *
data = pack('<BQ', 1, password)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_X_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[alice_keypair])#, *[payer_loaded_account])

In [70]:
# deposite y
# time.sleep(30)
from struct import *
data = pack('<BQ', 1 , password)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_Y_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[bob_keypair])#, *[payer_loaded_account])

In [76]:
# withdraw alice
from struct import *
data = pack('<BQ', 2, password)

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_Y_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaulty, is_signer=False, is_writable=True),
          AccountMeta(pubkey=alice_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[alice_keypair])#, *[payer_loaded_account])

In [88]:
a = http_client.get_account_info(alice_Y_token_account)
a

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 87155767},
  'value': {'data': ['bI0IOY3QS0Ah6U7ad9K8yEJEHxZ6seO9Gfk0hwWVj4SWVwf+b5vaPlJ9izp2ob5UA1xCN+4/XabVFd1Gf3PM/i0AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA',
    'base64'],
   'executable': False,
   'lamports': 2039280,
   'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
   'rentEpoch': 201}},
 'id': 143}

In [77]:
# withdraw bob
from struct import *
data = pack('<BQ', 2, password) 

import struct
tx = Transaction()
tx_instruction = TransactionInstruction(
    keys=[AccountMeta(pubkey=escrow_address, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_X_token_account, is_signer=False, is_writable=True), # x_a info
          AccountMeta(pubkey=vaultx, is_signer=False, is_writable=True),
          AccountMeta(pubkey=bob_pubkey, is_signer=True, is_writable=False),
          AccountMeta(pubkey=TOKEN_PROGRAM_ID, is_signer=False, is_writable=False),
         ],
    program_id=program_id,
    data=  data,
)

tx = tx.add(tx_instruction)


transaction_results = http_client.send_transaction(tx, *[bob_keypair])#, *[payer_loaded_account])

In [ ]:
# Trouble shooting
# {'code': -32602, 'message': 'invalid transaction: index out of bounds'}: probabily signed with different account in send_transaction function
# debit credit, pobably you need to airdrop solana to the account
# Cross-program invocation with unauthorized signer or writable account: the signer has not signed correctly in send_transaction
# Failed to serialize or deserialize account data it was where I wanted to write from account data to a struct and account data length was more than strust length (remember struct length shuld be in bytes not bits)